In [3]:
import math
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta
import yfinance as yf
pd.options.mode.chained_assignment = None

In [ ]:
acao = "PETR4.SA"

inicio = "2015-12-31"
fim = "2024-04-10"
dados_acao = yf.download(acao, inicio, fim)
print(dados_acao)